In [31]:
# Import Python packages
import pandas as pd
import os
import json
import pickle
import numpy as np
import time
import sys
# import cachetools
# Import Snowflake modules
from snowflake.snowpark import Session, DataFrame
from snowflake.snowpark import functions as F
from snowflake.snowpark import types as T
from snowflake.snowpark import Row, Column
import pandas as pd
from snowflake.snowpark.version import VERSION

In [32]:
# Get account credentials from a json file
with open("../connection.json") as f:
    data = json.load(f)
    username = data["user"]
    password = data["password"]
    account = data["account"]
    database = data["database"]
    schema = data["schema"]
    warehouse = data["warehouse"]

# Specify connection parameters
connection_parameters = {
    "account": account,
    "user": username,
    "password": password,
    "warehouse": warehouse,
    "database": database,
    "schema": schema,
}

# Create Snowpark session
session = Session.builder.configs(connection_parameters).create()
# Add a query tag to the session.
session.query_tag = {"origin":"sf_sit-is", 
                     "name":"spcs_call_center", 
                     "version":{"major":1, "minor":0},
                     "attributes":{"is_quickstart":1, "source":"notebook"}}
snowflake_environment = session.sql('SELECT current_user(), current_version()').collect()

print(session.get_current_warehouse())
snowpark_version = VERSION

# Current Environment Details
print('\nConnection Established with the following parameters:')
print('User                        : {}'.format(snowflake_environment[0][0]))
print('Role                        : {}'.format(session.get_current_role()))
print('Database                    : {}'.format(session.get_current_database()))
print('Schema                      : {}'.format(session.get_current_schema()))
print('Warehouse                   : {}'.format(session.get_current_warehouse()))
print('Snowflake version           : {}'.format(snowflake_environment[0][1]))
print('Snowpark for Python version : {}.{}.{}'.format(snowpark_version[0],snowpark_version[1],snowpark_version[2]))

"XS_WH"

Connection Established with the following parameters:
User                        : PRAJ
Role                        : "SPCS_PSE_ROLE"
Database                    : "PR_LLMDEMO"
Schema                      : "PUBLIC"
Warehouse                   : "XS_WH"
Snowflake version           : 8.1.0
Snowpark for Python version : 1.9.0


In [33]:
generated_transcripts_df = session.table("ALL_CLAIMS_RAW")

In [34]:
generated_transcripts_df.count()

96

In [35]:
generated_transcripts_df.limit(3).to_pandas()

,DATETIME,AUDIOFILE,TYPES,CONVERSATION,PRESIGNED_URL_PATH,DURATION
0,2023-11-11,audiofile1.mp3,Dispute Resolution,"Hello, this is Emily from AutoAssure Insurance...",https://sfc-prod3-ds1-16-customer-stage.s3.us-...,218.254271
1,2023-11-11,audiofile2.mp3,Dispute Resolution,"Hello, this is James from AutoAssure Insurance...",https://sfc-prod3-ds1-16-customer-stage.s3.us-...,197.705486
2,2023-11-11,audiofile3.mp3,Dispute Resolution,"Hello, this is Sarah from AutoAssure Insurance...",https://sfc-prod3-ds1-16-customer-stage.s3.us-...,75.172382


### Grant Role to access Cortex Functions
Only the below command should be executed as Accountadmin and rest of the code using custom role you have created(SPCS_PSE_ROLE)

```sql
USE ROLE accountadmin;
GRANT DATABASE ROLE SNOWFLAKE.CORTEX_USER TO ROLE  SPCS_PSE_ROLE
```

### Custom Prompt

In [36]:
session.sql("create stage if not exists udf").collect()

[Row(status='UDF already exists, statement succeeded.')]

In [37]:
# @F.udf(name="get_qa_prompt_zeroshort", session=session, packages=['scikit-learn==1.1.1', 'pandas'], is_permanent=True, stage_location="@udf", replace=True, input_types=[T.PandasSeriesType(T.StringType())], return_type=T.PandasSeriesType(T.StringType()))
# def get_qa_prompt(query_series:pd.Series):
#     response_series =  query_series.map(lambda query: f"""
#   [INST]We need to analyze the text of the customer support transcript in the <transcript></transcript> tag.
#    <transcript>
#      {query}
#    </transcript>
#   Provide a  response using the following format, no preamble. Extract the name of the Representative name from the transcript 
#   and replace that name for Representative and extract Customer name from the trasnscript 
#   replace that name for Customer for  building the conversation below :
#  * Representative : text of the Representative
#  * Customer  : text of the customer

#  This patterns should be a conversation between Representative and Customer conversation. Break the entire transcript into a conversation between 
#  Representative and the Customer. 
#    Don't provide any explanations. Don't include the sentense Sure, here is the conversation between:
# [/INST]""")
#     return response_series

In [38]:
@F.udf(name="get_qa_prompt_oneshort", session=session, packages=['scikit-learn==1.1.1', 'pandas'], is_permanent=True, stage_location="@udf", replace=True, input_types=[T.PandasSeriesType(T.StringType())], return_type=T.PandasSeriesType(T.StringType()))
def get_qa_prompt_oneshort(query_series:pd.Series):
    response_series =  query_series.map(lambda query: f"""
                                        
<s>[INST]
<<SYS>>
You are a expert in analyzing customer transcripts and tasked with answering questions about an article.  
Please ensure that your responses are socially unbiased and positive in nature.
If you don't know the answer, please don't share false information.

Output answer in the following format and no preamble. :
                                        * <Representative> : text of the Representative
                                        * <Customer>  : text of the customer
Extract the name of the Representative name from the transcript 
and replace that name for <Representative> and extract Customer name from the trasnscript replace that with <Customer> for creating the conversation. 

Below is one example output for the conversation:   
                                                                    
Representative: Jason
Customer: Michelle Turner

Jason: Hi, Michelle. My name is Jason from AutoAssure Insurance. How may I assist you today?
Michelle: Hi, Jason. I'm calling regarding a dispute I have with a recent claim.
Jason: Of course, Michelle. Can you please provide me with your claim number so I can access your file?
Michelle: Certainly, it's CLM876543.
Jason: Thank you, Michelle. Let me retrieve your claim information. While I'm doing that, could you explain the nature of the dispute?
Michelle: Of course, Jason. I believe there's been a misunderstanding in the assessment of the damages to my vehicle.
Jason: I appreciate you providing details, Michelle. I'll review your claim thoroughly and work towards a resolution.
Michelle: Thank you, Jason. I hope we can reach a fair resolution.
Jason: We'll do our best, Michelle. I'll keep you updated on the progress. Have a good day!
                                                                              
<</SYS>>                                                                                                             
We need to analyze the text of the transcript found in <transcript> </transcript> tag as conversation between representative and the customer. Convert the transcript in the format mentioned in the example. 
Extract the name of the Representative name from the transcript and replace that name for <Representative> and extract Customer name from the trasnscript replace that with <Customer> while creating the conversation.                                       
Do not include "Sure, here is the conversation between in the output. Never say thank you, that you are happy to help, that you are an AI agent, etc. Just answer directly". 
<transcript>
     {query}
</transcript> 


[/INST] Answer: Sure, I would be happy to help! """                                        
    )

    return response_series

In [39]:
# prompted_snowdf = generated_transcripts_df.with_column("PROMPT", F.call_udf("get_prompt", F.col("CONVERSATION")))

In [40]:
# zeroshort_prompted_snowdf = generated_transcripts_df.with_column("Zeroshort_PROMPT", F.call_udf("get_qa_prompt_zeroshort", F.col("CONVERSATION")))

In [41]:
oneshort_prompted_snowdf = generated_transcripts_df.with_column("Oneshort_PROMPT", F.call_udf("get_qa_prompt_oneshort", F.col("CONVERSATION")))

In [42]:
oneshort_prompted_snowdf.limit(2).to_pandas()

,DATETIME,AUDIOFILE,TYPES,CONVERSATION,PRESIGNED_URL_PATH,DURATION,ONESHORT_PROMPT
0,2023-11-11,audiofile1.mp3,Dispute Resolution,"Hello, this is Emily from AutoAssure Insurance...",https://sfc-prod3-ds1-16-customer-stage.s3.us-...,218.254271,\n \n<s...
1,2023-11-11,audiofile2.mp3,Dispute Resolution,"Hello, this is James from AutoAssure Insurance...",https://sfc-prod3-ds1-16-customer-stage.s3.us-...,197.705486,\n \n<s...


In [43]:
# zeroshort_prompted_snowdf.write.mode("overwrite").save_as_table("ALLCLAIMS_QA_Zeroshort_PROMPTED")
oneshort_prompted_snowdf.write.mode("overwrite").save_as_table("AudioConversation_With_Oneshort_PROMPT")

In [44]:
# zeroshort_prompted_snowdf = session.table("ALLCLAIMS_QA_Zeroshort_PROMPTED")

In [45]:
oneshort_prompted_snowdf = session.table("AudioConversation_With_Oneshort_PROMPT")

In [46]:
# 
# oneshort_prompted_snowdf.show(2)

In [47]:
# response_table_name = "TRANSCRIPT_INFO_EXTRACTED"

In [48]:
oneshort_info_extracted_snowdf = oneshort_prompted_snowdf.with_column("EXTRACTED_INFO", F.call_builtin("SNOWFLAKE.CORTEX.COMPLETE", F.lit('llama2-70b-chat'), F.col("ONESHORT_PROMPT")))

In [49]:
%%time
# oneshort_test_info_extracted_snowdf = oneshort_test_snowdf.with_column("EXTRACTED_INFO", F.call_builtin("snowflake.ml.COMPLETE", F.lit('llama2-70b-chat'), F.col("ONESHORT_PROMPT")))

CPU times: user 3 µs, sys: 2 µs, total: 5 µs
Wall time: 6.91 µs


In [50]:

# oneshort_info_extracted_snowdf.select('EXTRACTED_INFO').show(2)

In [51]:
%%time
oneshort_info_extracted_snowdf.write.mode('overwrite').save_as_table("TRANSCRIPT_INFO_EXTRACTED_QA")


CPU times: user 11 ms, sys: 2.49 ms, total: 13.4 ms
Wall time: 14.5 s


## Loading AUDIO_CLAIMS_EXTRACTED_INFO

In [52]:
# session.sql("truncate table AUDIO_CLAIMS_EXTRACTED_INFO").collect()

[Row(status='Statement executed successfully.')]

In [ ]:
#creating the AUDIO_CLAIMS_EXTRACTED_INFO table

session.sql('''
create or replace TABLE AUDIO_CLAIMS_EXTRACTED_INFO 
    (
        DATETIME DATE,
        AUDIO_FILE_NAME VARCHAR(100),
        AUDIO_FULL_FILE_PATH VARCHAR(16777216),
        RAW_CONVERSATION VARCHAR(16777216),
        PROMPTED_CONVERSATION VARCHAR(16777216),
        DURATION FLOAT,
        CALL_DETAILS VARIANT,
        CALL_SUMMARY VARCHAR(16777216),
        CALL_SENTIMENT FLOAT,
        REPONSE_GIVEN FLOAT
    )
''').collect()

In [53]:
%%time
session.sql(f"""
insert into AUDIO_CLAIMS_EXTRACTED_INFO(
                                        datetime,
                                        audio_file_name,       
                                        audio_full_file_path,
                                        raw_conversation,
                                        prompted_conversation,
                                        duration,
                                        call_details,
                                        call_summary,
                                        call_sentiment
                                    )
with audio_details as
(
    select datetime, 
            audiofile, 
            presigned_url_path, 
            conversation,
            extracted_info,
            duration,
            parse_json(SNOWFLAKE.CORTEX.COMPLETE('llama2-70b-chat',concat('<s>[INST] Analyze the audio transcripts found between the tags <question></question> tags and generate only the requested json output. Do not include any other language before or after the requested output.Include any policy number or claim number with values like policy number POL_NotAvailable or claim number CL_NotAvailable. Do not include the prompt. Output should only be json and if  ClaimNumber is not found then assign the value as NotFound in double quotes and if there is no PolicyNumber found then assign the value as NotFound in double quotes. 
            Provide a JSON response using the following format, no preamble. Collision with animal cannot be a intent. Below is the JSON that should be the output:
            {{ Representative: string,
            Customer:string, 
            ClaimNumber : string, 
            PolicyNumber : string,
            CallIntent:string,
            CallToAction:string,
            Issue:string,
            Resolution:string,
            NextSteps:string,
            ModeofUpdate:string,
            PurposeOfCall:string,
            ResponseMode:string,
            FirstCallResolution:float,
            CallQuality: string,
            Net Promoter Score :INT}}. Intent of the call should be in 2 words. Provide if it was a first call resolution as a score values ranging between 0.01 and 0.99.
            Call Quality involves monitoring and evaluating the quality of interactions based on communication skills, adherence to protocols, and overall customer handling and values will be Good,Average,Poor. 
            Net Promoter Score value between 10 -100 and also consider the resolution response time and skill of the Representative in handling the issue.
            
            Below pattern of conversations is not a first call resolution :
            Owen: Hello, this is Owen from AutoAssure Insurance. How can I assist you today?
            Jessica: Hi, Owen. My name is Jessica Turner, and I am calling to check the status of my insurance claim for a recent car accident.
            Owen: I am here to help, Jessica. Can you please provide me with your policy number so I can access your information?
            Jessica: It is POL901234.
            Owen: Thank you, Jessica. Let me pull up the details of your claim. While I am doing that, is there something specific you are unclear about regarding the status?
            Jessica: Yes, Owen. I had an accident two weeks ago, and I have not received any updates on the claim.
            Owen: I understand your concern, Jessica. I will investigate the status of your claim and provide you with an update. You will receive an email shortly.
            Jessica: Yes, Owen. How soon can I expect to hear back about the status of my claim?
            Owen: I appreciate your patience, Jessica. Our claims department is actively working on your case, and you can expect an update within the next 48 hours.
            Jessica: Thank you, Owen. I will be waiting for the update. You haveve been very helpful.
            Owen: It is my pleasure, Jessica. If you have any more questions or concerns, feel free to contact us.
            Jessica: Goodbye.
            
            Only provide JSON in your response. Dont provide any explanations.Do not prefix "Here is the JSON response for the conversation" for the json data generated. Value for the Claim Number should be NotFound in double quotes if Claim Number or Policy Number is not found in the text between the <question></question> for all rows.','<question>'
            ,EXTRACTED_INFO,'</question> [/INST] Answer: Here is the JSON response for the conversation')
            )
            ) as call_details 
            ,SNOWFLAKE.CORTEX.SUMMARIZE(concat('Provide the summary for the conversation in <con> </con> tag.Generate only the requested output. 
            Do not include any other language before or after the requested output.Also include any policy number or claim number if found the conversation. Do not include the prompt. Dont provide any explanations.
            <con>',CONVERSATION,'</con>')) as call_summary
            , SNOWFLAKE.CORTEX.SENTIMENT(CONVERSATION) as call_sentiment
from TRANSCRIPT_INFO_EXTRACTED_QA
)
select * from audio_details

""").collect()

CPU times: user 7.72 ms, sys: 4.38 ms, total: 12.1 ms
Wall time: 15.8 s


[Row(number of rows inserted=96)]

In [54]:
session.table("AUDIO_CLAIMS_EXTRACTED_INFO").limit(3).to_pandas()

,DATETIME,AUDIO_FILE_NAME,AUDIO_FULL_FILE_PATH,RAW_CONVERSATION,PROMPTED_CONVERSATION,DURATION,CALL_DETAILS,CALL_SUMMARY,CALL_SENTIMENT,REPONSE_GIVEN
0,2023-11-11,audiofile1.mp3,https://sfc-prod3-ds1-16-customer-stage.s3.us-...,"Hello, this is Emily from AutoAssure Insurance...",Representative: Emily\nCustomer: Rachel Ander...,218.254271,"{\n ""CallIntent"": ""Dispute Resolution"",\n ""C...","Sure, here is the summary of the conversation:...",0.849523,NaN
1,2023-11-11,audiofile2.mp3,https://sfc-prod3-ds1-16-customer-stage.s3.us-...,"Hello, this is James from AutoAssure Insurance...","I apologize, but I cannot provide a response ...",197.705486,"{\n ""CallIntent"": ""Dispute Resolution"",\n ""C...","Sure, here is the summary of the conversation:...",0.862604,NaN
2,2023-11-11,audiofile3.mp3,https://sfc-prod3-ds1-16-customer-stage.s3.us-...,"Hello, this is Sarah from AutoAssure Insurance...",Representative: Sarah\nCustomer: Alex Johnson...,75.172382,"{\n ""CallIntent"": ""Dispute"",\n ""CallQuality""...","Sure, here is the summary of the conversation:...",0.807768,NaN


In [55]:
# info_extracted_snowdf = prompted_snowdf.with_column("EXTRACTED_INFO", F.call_builtin("snowflake.ml.COMPLETE", F.lit('llama2-7b-chat'), F.col("PROMPT")))

In [56]:
%%time
# info_extracted_snowdf.write.mode("overwrite").save_as_table(response_table_name)

CPU times: user 1e+03 ns, sys: 1 µs, total: 2 µs
Wall time: 4.05 µs


## Loading the StreamlitApp Table
This is the source table for the Streamlit App for Text2SQL option

In [57]:
session.sql(f'''

CREATE OR REPLACE TABLE StreamlitAppTable
As
select datetime , audio_file_name,
duration,
call_details:CallToAction::string as CallToAction
,call_details:ClaimNumber::string as ClaimNumber
,call_details:Customer::string as Customer
,call_details:CallIntent::string as Intent
,call_details:Issue::string as Issue
,call_details:PolicyNumber::string as PolicyNumber
,call_details:PurposeOfCall::string as PurposeOfCall
,call_details:Representative::string as Representative
,call_details:Resolution::string as Resolution
,call_details:ResponseMode::string as ResponseMode
,call_details:ModeofUpdate::string as ModeofUpdate
,call_details:NextSteps::string as NextSteps
,IFF(call_sentiment>0.7,'Positive','Negative') as CallSentiment
,IFF(call_details:FirstCallResolution>0.5,'Yes','No') as FirstCallResolution
from
Audio_Claims_Extracted_info
            
            ''').collect()

[Row(status='Table STREAMLITAPPTABLE successfully created.')]

## Loading the Audio Embedding Table
This is used by the Chatbot part of the app

In [ ]:
session.sql('''
        create or replace table Audio_Call_Embedding
        as
        with sub
        as
        (
        select 
                datetime as CallDate,
                audio_file_name as AudioFileName,
                call_details:CallToAction as CallToAction,
                call_details:ClaimNumber::string as ClaimNumber ,
                call_details:PolicyNumber::string as PolicyNumber,
                call_details:Representative::string as AgentName,
                call_details:Customer::string as CustomerName,
                call_details:CallIntent::string as CallIntent,
                call_details:Issue as Issue,
                call_details:ModeofUpdate as ModeofUpdate,
                call_details:NextSteps as NextSteps,
                call_details:Resolution as Resolution,
                IFF(call_details:FirstCallResolution>0.5,'Yes','No') as FristCallResolution,
            IFF(call_sentiment >0.72,'Positive','Negative') as CallSentiment,
            substr(call_summary,charindex(':',call_summary,1)+1) as CallSummary
        
        from Audio_Claims_Extracted_info
        )
        select OBJECT_CONSTRUCT(*) as audio_data, SNOWFLAKE.CORTEX.embed_text_768('e5-base-v2' ,
        audio_data::string) as ct_embedding from sub ;
                    
            ''').collect()

[Row(status='Table AUDIO_CALL_EMBEDDING successfully created.')]

#### AutoClaim and PolicytDetails Master Table 

In [ ]:
session.sql(''' 
CREATE or replace TABLE PolicyDetails (
    PolicyID                INT IDENTITY,
    PolicyNumber            VARCHAR(50) UNIQUE,
    CustomerID              INT,
    PolicyType              VARCHAR(50),
    CoverageAmount          DECIMAL(10, 2),
    PremiumAmount           DECIMAL(8, 2),
    StartDate               DATE,
    EndDate                 DATE,
    RenewalDate             DATE,
    PolicyStatus            VARCHAR(20),
    PaymentStatus           VARCHAR(20),
    Beneficiaries           VARCHAR(255),
    CoverageDetails         TEXT,
    RidersEndorsements      TEXT,
    InsuredItemsPersons     TEXT,
    PolicyholderDetails     TEXT,
    AgentBrokerDetails      TEXT,
    Deductibles             DECIMAL(8, 2),
    Exclusions              TEXT,
    ClaimsHistory           TEXT,
    UnderwritingDetails     TEXT,
    PolicyTermsConditions   TEXT
); ''').collect()




In [ ]:
session.sql(''' 
INSERT INTO PolicyDetails (     
PolicyNumber         
,CustomerID           
,PolicyType           
,CoverageAmount       
,PremiumAmount        
,StartDate            
,EndDate              
,RenewalDate          
,PolicyStatus         
,PaymentStatus        
,Beneficiaries        
,CoverageDetails      
,RidersEndorsements   
,InsuredItemsPersons  
,PolicyholderDetails  
,AgentBrokerDetails   
,Deductibles          
,Exclusions           
,ClaimsHistory        
,UnderwritingDetails  
,PolicyTermsConditions) VALUES
( 'POL001', 1001, 'Auto', 500000, 1000, '2022-01-01', '2023-01-01', '2022-12-31', 'Active', 'Paid', 'John Doe', 'Basic coverage for vehicle insurance.', 'None', 'John Doe, Jane Doe', 'John Doe, 123 Insurance Inc.', 'John Doe, Jane Doe', 100, 'None', 'No claims made yet.', 'Standard underwriting process completed.', 'Standard terms and conditions apply.'),
( 'POL002', 1002, 'Auto', 100000, 500, '2022-02-01', '2023-02-01', '2023-01-31', 'Active', 'Due', 'Jane Doe', 'Basic coverage for health insurance.', 'Critical Illness Rider', 'Jane Doe, Family Members', 'Jane Doe, 456 Insurance Services', 'Jane Doe, Family Members', 50, 'Pre-existing conditions', 'No claims made yet.', 'Medical examination completed.', 'Standard terms and conditions apply.'),
( 'POL010', 1010, 'Auto', 25000, 300, '2022-10-01', '2023-10-01', '2023-09-30', 'Active', 'Paid', 'Alice Smith', 'Basic coverage for auto insurance.', 'Roadside Assistance Rider', 'Alice Smith, Vehicle Details', 'Alice Smith, XYZ Insurance Agency', 'Alice Smith, Vehicle Details', 75, 'Accidents caused by negligence', 'No claims made yet.', 'Vehicle inspection completed.', 'Standard terms and conditions apply.'),
( 'POL934514', 1007, 'Auto', 500000, 1000, '2023-08-01', '2024-08-01', '2024-07-25', 'Active', 'Paid', 'Mike S', 'Full coverage for vehicle insurance.', 'None', 'Mike S', 'Mike S, 456 Insurance Inc.', 'Mike S, 
Jane S', 100, 'None', '2 claims made so far.', 'Standard underwriting process completed.', 'Standard terms and conditions apply.')

''').collect()


In [ ]:
session.sql('''
CREATE or replace TABLE AutoClaims (
    ClaimID varchar(50) ,
    PolicyID INT,
    ClaimDate DATE,
    ClaimAmount DECIMAL(10, 2),
    ClaimDescription TEXT,
    ClaimStatus VARCHAR(50)
)
''').collect()

In [ ]:
session.sql('''
INSERT INTO AutoClaims (ClaimID,PolicyID, ClaimDate, ClaimAmount, ClaimDescription, ClaimStatus)
VALUES
    ('CLM234567',4, '2023-09-15', 5000, 'Vehicle collision at an intersection.', 'Pending'),
    ('CLM876543',5, '2023-04-20', 1200, 'Minor fender bender in a parking lot.', 'Approved'),
    ('CLM567890',6, '2023-10-10', 8000, 'Total loss due to a major accident on the highway.', 'Denied'),
    ('CLM345678901',7, '2023-09-15', 5000, 'Vehicle Damage', 'Pending')
''').collect()

In [59]:
session.close()

In [60]:
# generated_transcripts_df = session.table(output_table_name)